# Fund Performance and Attribution

In [29]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [30]:
ltcm = pd.read_excel('/Users/dylanpan/Documents/FINM 36700 Portfolio and Risk Managemenet/Homework/Data/ltcm_exhibits_data.xlsx', sheet_name='Exhibit 2', skiprows=2, index_col=0)
ltcm = ltcm.iloc[1:-4, :] # remove null rows at the top and bottom
ltcm.rename(columns={'Gross Monthly Performancea': 'Gross Monthly Performance',  'Net Monthly Performanceb': 'Net Monthly Performance'}, inplace=True) # correct column names
ltcm.index = pd.to_datetime(ltcm.index)
ltcm.index = ltcm.index.to_period('M').to_timestamp('M') # convert index to month-end
ltcm.head()

,Fund Capital ($billions),Gross Monthly Performance,Net Monthly Performance,Index of Net Performance
1994-03-31,1.1,-0.011,-0.013,0.99
1994-04-30,1.1,0.014,0.008,1.00
1994-05-31,1.2,0.068,0.053,1.05
1994-06-30,1.2,-0.039,-0.029,1.02
1994-07-31,1.4,0.116,0.084,1.10


In [31]:
spy = pd.read_excel('/Users/dylanpan/Documents/FINM 36700 Portfolio and Risk Managemenet/Homework/Data/spy_data.xlsx', sheet_name='total returns', index_col=0)
spy.index = pd.to_datetime(spy.index)
spy.index = spy.index.to_period('M').to_timestamp('M') # convert index to month-end
spy = spy.loc[ltcm.index] # align spy index with ltcm index
spy['RF'] = spy['^IRX']
spy.drop(columns=['^IRX'], inplace=True)
spy['excess_return'] = spy['SPY'] - spy['RF']
spy.head()

,SPY,RF,excess_return
1994-03-31,-0.047397,0.002892,-0.050288
1994-04-30,0.011212,0.003217,0.007996
1994-05-31,0.015939,0.003475,0.012464
1994-06-30,-0.029332,0.003450,-0.032782
1994-07-31,0.032326,0.003558,0.028768


# 1. Summary stats

In [32]:
def calculate_summary_stats(returns, Period = 12):
    mean_return = returns.mean() * Period
    volatility = returns.std() * np.sqrt(Period)
    sharpe_ratio = mean_return / volatility

    skewness = returns.skew() # dimensionless
    kurtosis = returns.kurtosis() # dimensionless
    fifth_quantile = returns.quantile(0.05)
    
    return pd.DataFrame({
        'Mean Return': mean_return,
        'Volatility': volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Skewness': skewness,
        'Kurtosis': kurtosis,
        '5th Quantile': fifth_quantile,
    }, index=[0])


In [33]:
# calculate excess returns for LTCM
ltcm_merged = ltcm.join(spy['RF'], how='left')
ltcm_merged['Gross Excess Return'] = ltcm_merged['Gross Monthly Performance'] - ltcm_merged['RF']
ltcm_merged['Net Excess Return'] = ltcm_merged['Net Monthly Performance'] - ltcm_merged['RF']
ltcm_merged.head()

,Fund Capital ($billions),Gross Monthly Performance,Net Monthly Performance,Index of Net Performance,RF,Gross Excess Return,Net Excess Return
1994-03-31,1.1,-0.011,-0.013,0.99,0.002892,-0.013892,-0.015892
1994-04-30,1.1,0.014,0.008,1.00,0.003217,0.010783,0.004783
1994-05-31,1.2,0.068,0.053,1.05,0.003475,0.064525,0.049525
1994-06-30,1.2,-0.039,-0.029,1.02,0.003450,-0.042450,-0.032450
1994-07-31,1.4,0.116,0.084,1.10,0.003558,0.112442,0.080442


In [34]:
gross_stats = calculate_summary_stats(ltcm_merged['Gross Excess Return'])
net_stats = calculate_summary_stats(ltcm_merged['Net Excess Return'])

print("LTCM Gross Excess Return Summary Statistics:")
print(gross_stats)

print("\nLTCM Net Excess Return Summary Statistics:")
print(net_stats)

LTCM Gross Excess Return Summary Statistics:
   Mean Return  Volatility  Sharpe Ratio  Skewness  Kurtosis  5th Quantile
0       0.2436    0.136238       1.78805 -0.288328  1.586681     -0.030305

LTCM Net Excess Return Summary Statistics:
   Mean Return  Volatility  Sharpe Ratio  Skewness  Kurtosis  5th Quantile
0     0.156883    0.111773       1.40359  -0.81087  2.927724       -0.0263


# 2. Compare to SPY

In [35]:
spy_stats = calculate_summary_stats(spy['excess_return'])
print("\SPY Excess Return Summary Statistics:")
print(spy_stats)

\SPY Excess Return Summary Statistics:
   Mean Return  Volatility  Sharpe Ratio  Skewness  Kurtosis  5th Quantile
0     0.154775    0.114073      1.356806 -0.406867 -0.388002     -0.049667


Gross performance clearly outperforms SPY, exhibiting a substantially higher mean excess return and a stronger Sharpe ratio, indicating superior risk-adjusted performance. However, this outperformance comes at the cost of higher volatility, suggesting greater overall risk. In contrast, net performance is much closer to SPY, with nearly comparable mean returns, volatility, and Sharpe ratios, indicating that after fees, LTCM’s advantage is significantly reduced.

The difference between gross and net performance, while not extreme, is still meaningful. The mean excess return declines from 0.24 to 0.16 and the Sharpe ratio drops from 1.78 to 1.40, highlighting the impact of management fees and trading costs. At the same time, volatility decreases from 0.13 to 0.11, suggesting slightly reduced risk but also diminished return efficiency. Overall, fees materially erode LTCM’s performance edge relative to SPY.

# 3. LFD

In [36]:
Y = ltcm_merged['Net Excess Return']
X = spy['excess_return']

model = sm.OLS(Y, sm.add_constant(X)).fit()

alpha = model.params['const'] * 12
beta = model.params['excess_return']
r_squared = model.rsquared

model_summary = pd.DataFrame({
    'Alpha (Annualized)': [alpha],
    'Beta_SPY': [beta],
    'R-squared': [r_squared]
})
model_summary

,Alpha (Annualized),Beta_SPY,R-squared
0,0.135076,0.140892,0.020676


Although LTCM shows a positive annualized alpha (~13%), the extremely low R-squared (~0.02) and small beta suggest weak explanatory power and little consistent performance beyond SPY. This indicates that LTCM’s apparent outperformance is not statistically strong or reliably different from SPY’s performance.

# 4. Nonlinear Exposure

$$
\tilde{r}^{LTCM}_t = \alpha + \beta_{\text{linear}}\tilde{r}^m_t + \beta_{\text{quad}}(\tilde{r}^m_t)^2 + \varepsilon_t
$$


In [38]:
Y = ltcm_merged['Net Excess Return']
X = pd.DataFrame({
    'Linear': spy['excess_return'],
    'Quadratic': spy['excess_return'] ** 2})

model = sm.OLS(Y, sm.add_constant(X)).fit()

alpha = model.params['const'] * 12
beta_linear = model.params['Linear']
beta_quadratic = model.params['Quadratic']
r_squared = model.rsquared

model_summary_nonlinear = pd.DataFrame({
    'Alpha (Annualized)': [alpha],
    'Beta Linear': [beta_linear],
    'Beta Quadratic': [beta_quadratic],
    'R-squared': [r_squared]
})
model_summary_nonlinear

,Alpha (Annualized),Beta Linear,Beta Quadratic,R-squared
0,0.162629,0.168741,-2.158255,0.027801


# 5. 
 
1. Does the quadratic factor increase explained variation?

No. The R-squared only increases slightly from about 0.0207 (linear model) to 0.0278 (quadratic model). This is a very small improvement, indicating that adding the quadratic term does not meaningfully increase the amount of LTCM return variation explained by the market. Market exposure remains weak overall.

2. Is LTCM long or short market options?

Quadratic beta is negative (≈ -2.158).
A negative quadratic coefficient implies convexity: LTCM deteriorates disproportionately more when market moves are large (either up or down)
This is consistent with short market options.

3. Is LTCM positively or negatively exposed to market volatility?

LTCM disproportionately becomes worse during extreme market move, regardless of the direction. Therefore, LTCM is negatively exposed to market volatility as having a negative quadratic beta.



# 6.

In [41]:
k1 = 0.03
k2 = -0.03
upmarket = np.maximum(spy['excess_return'] - k1, 0)
downmarket = np.maximum(k2 - spy['excess_return'], 0)

X = pd.DataFrame({
    'Market': spy['excess_return'],
    'Upmarket': upmarket,
    'Downmarket': downmarket
})

model = sm.OLS(Y, sm.add_constant(X)).fit()
alpha = model.params['const'] * 12
beta_market = model.params['Market']
beta_upmarket = model.params['Upmarket']
beta_downmarket = model.params['Downmarket']
r_squared = model.rsquared

model_summary_asymmetric = pd.DataFrame({
    'Alpha (Annualized)': [alpha],
    'Beta Market': [beta_market],
    'Beta Upmarket': [beta_upmarket],
    'Beta Downmarket': [beta_downmarket],
    'R-squared': [r_squared]
})
model_summary_asymmetric

,Alpha (Annualized),Beta Market,Beta Upmarket,Beta Downmarket,R-squared
0,0.109438,0.434499,-0.721876,1.042253,0.048607


# 7.

1. Is LTCM long or short the call-like and put-like factors?


- For call-like factor max(spy - k1, 0), the upmarket beta = -0.72, which is negative. This means LTCM performs worse when the market experiences large positive moves. Therefore, LTCM is short the call-like factor.

- For put-like factor max(k2 - spy, 0), the downmarket beta = +1.04, which is positive. This means LTCM performs better when the market experiences large negative move. Therefore, LTM is long the put-like factor.
 

2. Which factor moves LTCM more?

The put-life factors moves LTCM more, given it has greater absolute beta.

3. What does this say about volatility exposure?

This regression shows that LTCM’s nonlinear behavior is not symmetric:

In big rallies, up beta is negative, indicating that LTCM loses money when the market jumps strongly upward. In big crashes, down beta is positive, indicating LTCM gains more money when market falls sharply. Therefore, It is NOT simply long or short volatility overall. Instead, it has asymmetric volatility exposure that it is harmed by large positive moves, and that it benefits from large negative moves.




